In [1]:
import pandas as pd
songs = pd.read_csv('../data/songs.csv')
streams = pd.read_csv('../data/streams1.csv')

In [12]:
print(songs.columns)
print(streams.columns)

Index(['id', 'track_id', 'artists', 'album_name', 'track_name', 'popularity',
       'duration_ms', 'explicit', 'danceability', 'energy', 'key', 'loudness',
       'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature', 'track_genre'],
      dtype='object')
Index(['user_id', 'track_id', 'listen_time'], dtype='object')


In [13]:
print(songs.info())
print(streams.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89741 entries, 0 to 89740
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                89741 non-null  int64  
 1   track_id          89741 non-null  object 
 2   artists           89740 non-null  object 
 3   album_name        89740 non-null  object 
 4   track_name        89740 non-null  object 
 5   popularity        89741 non-null  int64  
 6   duration_ms       89741 non-null  int64  
 7   explicit          89741 non-null  bool   
 8   danceability      89741 non-null  float64
 9   energy            89741 non-null  float64
 10  key               89741 non-null  int64  
 11  loudness          89741 non-null  float64
 12  mode              89741 non-null  int64  
 13  speechiness       89741 non-null  float64
 14  acousticness      89741 non-null  float64
 15  instrumentalness  89741 non-null  float64
 16  liveness          89741 non-null  float6

In [15]:
#get a glance at the data
print(streams.head(5))
print(songs.head(5))

   user_id                track_id          listen_time
0    26213  4dBa8T7oDV9WvGr7kVS4Ez  2024-06-25 17:43:13
1     6937  4osgfFTICMkcGbbigdsa53  2024-06-25 07:26:00
2    21407  2LoQWx41KeqOrSFra089YS  2024-06-25 13:25:26
3    47146  7cfG5lFeJWEgpSnubt4O4W  2024-06-25 18:17:50
4    38594  6tilCYbheGMHo3Hw4F22hF  2024-06-25 17:33:21
   id                track_id                 artists  \
0   0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1   1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2   2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3   3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4   4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...   
4                                        

In [2]:
# Prepare data
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date
merged_data = streams.merge(songs, on='track_id', how='left')

# Compute each KPI
# KPI 1: Daily Genre Listen Count
genre_listen_count = merged_data.groupby(['listen_date', 'track_genre']).size().reset_index(name='listen_count')

# KPI 2: Average Listening Duration per Genre per Day
merged_data['duration_seconds'] = merged_data['duration_ms'] / 1000
avg_duration = merged_data.groupby(['listen_date', 'track_genre'])['duration_seconds'].mean().reset_index(name='average_duration')

# KPI 3: Daily Genre Popularity Index
total_listens = merged_data.groupby('listen_date').size().reset_index(name='total_listens')
genre_listen_count = genre_listen_count.merge(total_listens, on='listen_date')
genre_listen_count['popularity_index'] = genre_listen_count['listen_count'] / genre_listen_count['total_listens']

# KPI 4: Most Popular Track per Genre per Day
most_popular_track = merged_data.groupby(['listen_date', 'track_genre', 'track_id']).size().reset_index(name='track_count')
most_popular_track = most_popular_track.sort_values(by=['listen_date', 'track_genre', 'track_count'], ascending=[True, True, False])
most_popular_track = most_popular_track.drop_duplicates(subset=['listen_date', 'track_genre'], keep='first').rename(columns={'track_id': 'most_popular_track_id'})

# Combine all KPIs into one DataFrame
final_kpis = genre_listen_count[['listen_date', 'track_genre', 'listen_count', 'popularity_index']]
final_kpis = final_kpis.merge(avg_duration, on=['listen_date', 'track_genre'])
final_kpis = final_kpis.merge(most_popular_track[['listen_date', 'track_genre', 'most_popular_track_id']], on=['listen_date', 'track_genre'])

In [3]:
final_kpis.head()

,listen_date,track_genre,listen_count,popularity_index,average_duration,most_popular_track_id
0,2024-06-25,acoustic,118,0.010400,215.379034,0WWuB1F1H4dr3Bdoe1vtHs
1,2024-06-25,afrobeat,124,0.010929,248.978960,1KME77F9mu2RQS8vo6JVwa
2,2024-06-25,alt-rock,119,0.010488,236.073731,0GO8y8jQk1PkHzS31d699N
3,2024-06-25,alternative,45,0.003966,202.768200,0YwBrYaPYYc8e18ZYkqhJc
4,2024-06-25,ambient,119,0.010488,255.296067,2tr4oclswJ6v3dfDlI01HD
